# 🛒 Retail Product Analysis – Databricks Simulated Retail Dataset

📌 1. Introduction

This project focuses on building an end-to-end retail analytics dashboard using Databricks SQL and Databricks Lakeview Dashboard.

The goal is to analyze product-level and customer-level sales behavior, extract meaningful insights, and demonstrate how Databricks can be used to process semi-structured JSON data at scale and turn it into interactive business analytics.

The dashboard answers core questions such as:

What are the best-selling products and categories?

How does revenue trend over time?

Which products customers buy the most per order?

What are customer spending patterns?

What are the price–quantity dynamics across products?


📦 2. Dataset Description

I used the public dataset from:

Databricks Marketplace

Dataset: databricks_simulated_retail_customer_data
Tables:

Table	Description
sales	Contains order-level product information (JSON format)
customers	Contains customer demographics
sales_orders	Order metadata (clickstream etc.)

Most product details (name, id, price, qty, currency, unit) are embedded in a JSON field inside sales.product.

Because the product column is JSON, the first step is to parse and flatten the struct.

In [0]:
%sql
SELECT * FROM databricks_simulated_retail_customer_data.v01.sales LIMIT 10;

customer_id,customer_name,product_name,order_date,product_category,product,total_price
17372531,"RAMSEY, SHELBERT",Ramsung EVO+ 256GB UHS-I microSDXC U3 Memory Card with Adapter (MB-MC256DA/AM),2019-10-15,Ramsung,"{""curr"":""USD"",""id"":""AVpiE9hhilAPnD_xAfSU"",""name"":""Cyber-shot DSC-RX100 V Digital Camera"",""price"":2798,""qty"":4,""unit"":""pcs""}",11192
58578517,"MILLER, LAWANDA Y",SP-FS52 Andrew Jones Designed Floorstanding Loudspeaker,2019-08-07,Sioneer,"{""curr"":""USD"",""id"":""AVpfHcah1cnluZ0-eQLY"",""name"":""Elite A-20 2-Channel Integrated Amplifier"",""price"":758,""qty"":3,""unit"":""pcs""}",2274
58578517,"MILLER, LAWANDA Y",Sioneer GM-D8601 Class D Mono Amplifier with Wired Bass Boost Remote,2019-08-07,Sioneer,"{""curr"":""USD"",""id"":""AVpgRiy2LJeJML43Lk7h"",""name"":""SP-FS52 Andrew Jones Designed Floorstanding Loudspeaker"",""price"":424,""qty"":7,""unit"":""pcs""}",2968
17372531,"RAMSEY, SHELBERT","Opple NakBook - 12 - Core m5 - 8 GB RAM - 512 GB flash storage - English""""",2019-10-15,Opple,"{""curr"":""USD"",""id"":""AVpf-2hGilAPnD_xlfDv"",""name"":""Ramsung EVO+ 256GB UHS-I microSDXC U3 Memory Card with Adapter (MB-MC256DA/AM)"",""price"":273,""qty"":2,""unit"":""pcs""}",546
17372531,"RAMSEY, SHELBERT","15.4 NakBook Pro with Touch Bar (Late 2016, Space Gray)",2019-10-15,Opple,"{""curr"":""USD"",""id"":""AVpfZaCp1cnluZ0-kDV9"",""name"":""15.4 NakBook Pro with Touch Bar (Late 2016, Space Gray)"",""price"":6714,""qty"":5,""unit"":""pcs""}",33570
17372531,"RAMSEY, SHELBERT","15.4 NakBook Pro with Touch Bar (Late 2016, Space Gray)",2019-10-15,Opple,"{""curr"":""USD"",""id"":""AVpfZaCp1cnluZ0-kDV9"",""name"":""15.4 NakBook Pro with Touch Bar (Late 2016, Space Gray)"",""price"":6714,""qty"":5,""unit"":""pcs""}",33570
17372531,"RAMSEY, SHELBERT",Cyber-shot DSC-RX100 V Digital Camera,2019-10-15,Rony,"{""curr"":""USD"",""id"":""AVpfWGrYLJeJML437hk2"",""name"":""Opple NakBook - 12 - Core m5 - 8 GB RAM - 512 GB flash storage - English"""",""price"":3553,""qty"":4,""unit"":""pcs""}",14212
58578517,"MILLER, LAWANDA Y",Elite A-20 2-Channel Integrated Amplifier,2019-08-07,Sioneer,"{""curr"":""USD"",""id"":""AVpgUl_U1cnluZ0-z3Gz"",""name"":""Sioneer GM-D8601 Class D Mono Amplifier with Wired Bass Boost Remote"",""price"":413,""qty"":7,""unit"":""pcs""}",2891
58578517,"MILLER, LAWANDA Y",Opple MD825AM/A Lightning to VGA Adapter for iPhones,2019-08-07,Opple,"{""curr"":""USD"",""id"":""AVpggL_W1cnluZ0-2Wfp"",""name"":""Ramsung J3 - Verizon Prepaid"",""price"":242,""qty"":6,""unit"":""pcs""}",1452
58578517,"MILLER, LAWANDA Y",Sioneer - Elite 7.2-Ch. Hi-Res 4K Ultra HD HDR Compatible A/V Home Theater Receiver - Black,2019-08-07,Sioneer,"{""curr"":""USD"",""id"":""AVz5wclz-jtxr-f30F66"",""name"":""Sioneer - Elite 7.2-Ch. Hi-Res 4K Ultra HD HDR Compatible A/V Home Theater Receiver - Black"",""price"":1749,""qty"":3,""unit"":""pcs""}",5247


🔧 3. Data Processing & Engineering Steps

Step 1: Inspect JSON product column

Step 2: Parse JSON into structured fields

Step 3: Flatten into a clean analysis table

In [0]:
%sql
-- Detect valid JSON by checking required keys
-- Only parse JSON when it is likely valid
CREATE OR REPLACE TEMP VIEW sales_products AS
SELECT
    customer_id,
    customer_name,
    order_date,
    product_category,
    total_price,

    CASE 
        WHEN product LIKE '%"id"%' 
         AND product LIKE '%"name"%' 
         AND product LIKE '%"price"%' 
         AND product LIKE '%"qty"%' 
         AND product LIKE '%"unit"%'
        THEN from_json(
                product,
                'struct<curr:string,id:string,name:string,price:double,qty:int,unit:string>'
             )
        ELSE NULL
    END AS p

FROM `databricks_simulated_retail_customer_data`.`v01`.`sales`;




-- Flatten parsed struct into separate columns
CREATE OR REPLACE TEMP VIEW sales_products_flat AS
SELECT
    customer_id,
    customer_name,
    order_date,
    product_category,
    total_price,
    p.id      AS product_id,
    p.name    AS product_name,
    p.price   AS unit_price,
    p.qty     AS quantity,
    p.unit    AS unit
FROM sales_products;


In [0]:
%sql
select * from sales_products_flat limit 10;

customer_id,customer_name,order_date,product_category,total_price,product_id,product_name,unit_price,quantity,unit
17372531,"RAMSEY, SHELBERT",2019-10-15,Ramsung,11192,AVpiE9hhilAPnD_xAfSU,Cyber-shot DSC-RX100 V Digital Camera,2798.0,4,pcs
58578517,"MILLER, LAWANDA Y",2019-08-07,Sioneer,2274,AVpfHcah1cnluZ0-eQLY,Elite A-20 2-Channel Integrated Amplifier,758.0,3,pcs
58578517,"MILLER, LAWANDA Y",2019-08-07,Sioneer,2968,AVpgRiy2LJeJML43Lk7h,SP-FS52 Andrew Jones Designed Floorstanding Loudspeaker,424.0,7,pcs
17372531,"RAMSEY, SHELBERT",2019-10-15,Opple,546,AVpf-2hGilAPnD_xlfDv,Ramsung EVO+ 256GB UHS-I microSDXC U3 Memory Card with Adapter (MB-MC256DA/AM),273.0,2,pcs
17372531,"RAMSEY, SHELBERT",2019-10-15,Opple,33570,AVpfZaCp1cnluZ0-kDV9,"15.4 NakBook Pro with Touch Bar (Late 2016, Space Gray)",6714.0,5,pcs
17372531,"RAMSEY, SHELBERT",2019-10-15,Opple,33570,AVpfZaCp1cnluZ0-kDV9,"15.4 NakBook Pro with Touch Bar (Late 2016, Space Gray)",6714.0,5,pcs
17372531,"RAMSEY, SHELBERT",2019-10-15,Rony,14212,null,null,null,null,null
58578517,"MILLER, LAWANDA Y",2019-08-07,Sioneer,2891,AVpgUl_U1cnluZ0-z3Gz,Sioneer GM-D8601 Class D Mono Amplifier with Wired Bass Boost Remote,413.0,7,pcs
58578517,"MILLER, LAWANDA Y",2019-08-07,Opple,1452,AVpggL_W1cnluZ0-2Wfp,Ramsung J3 - Verizon Prepaid,242.0,6,pcs
58578517,"MILLER, LAWANDA Y",2019-08-07,Sioneer,5247,AVz5wclz-jtxr-f30F66,Sioneer - Elite 7.2-Ch. Hi-Res 4K Ultra HD HDR Compatible A/V Home Theater Receiver - Black,1749.0,3,pcs


❓4. Key Business Questions (Corrected)

This notebook answers the following business questions:

Q1. What are the top-selling products by quantity?

Q2. Which products generate the highest revenue?

Q3. How does revenue trend over time?

Q4. What is the price–quantity relationship across products?

Q5. Which products have the highest quantity per order? (bulk-buy behavior)

Each business question corresponds to one SQL cell and a recommended visualization.

In [0]:
%sql
-- Question 1: Which product categories generate the highest revenue?
-- We sum total_price grouped by product_category.

SELECT
    product_category,
    SUM(total_price) AS total_revenue
FROM sales_products_flat
GROUP BY product_category
ORDER BY total_revenue DESC;


product_category,total_revenue
Rony,947658
Sioneer,640831
Opple,581853
Ramsung,474119
Zamaha,70534
Reagate,21031


Databricks visualization. Run in Databricks to view.

### 📊 Category Revenue Analysis  
This chart shows which product categories contribute the most revenue.  
High-revenue categories indicate core business drivers and consumer preferences.  


In [0]:
%sql
-- Question 2: Which products generate the highest revenue?
-- We group by product_name and sum the total_price.

SELECT
    product_name,
    SUM(total_price) AS revenue
FROM sales_products_flat
GROUP BY product_name
ORDER BY revenue DESC
LIMIT 20;


product_name,revenue
null,644136
Cyber-shot DSC-RX100 V Digital Camera,576268
Sioneer - Elite 7.2-Ch. Hi-Res 4K Ultra HD HDR Compatible A/V Home Theater Receiver - Black,354804
Ramsung - 960 Pro 1TB Internal PCI Express 3.0 x4 (NVMe 1.1) Solid State Drive,348344
Elite A-20 2-Channel Integrated Amplifier,149888
Rony STRDN1070 7.2-channel AV Receiver w/ Bluetooth,116390
"15.4 NakBook Pro with Touch Bar (Late 2016, Space Gray)",67140
21.5 iNak with Retina 4K Display (Mid 2017),65568
128GB iPod touch (Gold) (6th Generation),59491
R-S300 Natural Sound Stereo Receiver,52863


Databricks visualization. Run in Databricks to view.

### 🥇 Top Revenue Products  
These are the best-performing products in terms of revenue.  
You can highlight "hero products" and evaluate if price or volume drives performance.  


In [0]:
%sql
-- Question 3: What is the daily revenue trend?
-- We aggregate revenue by order_date.

SELECT
    order_date,
    SUM(total_price) AS daily_revenue
FROM sales_products_flat
GROUP BY order_date
ORDER BY order_date;


order_date,daily_revenue
2019-08-02,98022
2019-08-05,36902
2019-08-06,206949
2019-08-07,14940
2019-08-12,223416
2019-08-13,70205
2019-08-16,53315
2019-08-19,35404
2019-08-20,52000
2019-08-21,14299


Databricks visualization. Run in Databricks to view.

### 📈 Daily Sales Trend  
This line chart reveals sales volatility, seasonal patterns, and peak business days.  
Great for identifying demand cycles and forecasting opportunities.  


In [0]:
%sql
-- Question 4: What is the relationship between pricing and quantity sold?
-- This helps understand whether expensive products sell less/more.

SELECT
    product_name,
    unit_price,
    SUM(quantity) AS total_quantity
FROM sales_products_flat
GROUP BY product_name, unit_price
ORDER BY total_quantity DESC;


product_name,unit_price,total_quantity
Ramsung - 960 Pro 1TB Internal PCI Express 3.0 x4 (NVMe 1.1) Solid State Drive,1744.0,185
BC-TRW W Series Battery Charger (Black),94.0,175
Cyber-shot DSC-RX100 V Digital Camera,2798.0,167
Sioneer - Elite 7.2-Ch. Hi-Res 4K Ultra HD HDR Compatible A/V Home Theater Receiver - Black,1749.0,166
Elite A-20 2-Channel Integrated Amplifier,758.0,163
Sioneer - 8x External USB 3.0 Quad-Layer Blu-ray Disc DL DVD±RW/CD-RW Drive - Silver,216.0,152
Rony STRDN1070 7.2-channel AV Receiver w/ Bluetooth,485.0,126
Ramsung EVO+ 256GB UHS-I microSDXC U3 Memory Card with Adapter (MB-MC256DA/AM),96.0,113
Cyber-shot DSC-RX100 V Digital Camera,982.0,111
Sioneer - Elite 7.2-Ch. Hi-Res 4K Ultra HD HDR Compatible A/V Home Theater Receiver - Black,614.0,105


Databricks visualization. Run in Databricks to view.

### 📉 Price vs Quantity Relationship  
This helps assess the elasticity of pricing.  
- High price + high quantity → Premium strong performers  
- Low price + high quantity → Volume-driven products  
- High price + low quantity → Niche or low-demand products  


In [0]:
%sql
-- Question 5: Which products have the highest average revenue per order?
-- Useful for identifying premium influencers.

SELECT
    product_name,
    AVG(total_price) AS avg_order_revenue,
    COUNT(*) AS order_count
FROM sales_products_flat
GROUP BY product_name
HAVING order_count > 10  -- filter out noise
ORDER BY avg_order_revenue DESC
LIMIT 20;


product_name,avg_order_revenue,order_count
null,21471.2,30
Cyber-shot DSC-RX100 V Digital Camera,18589.290322580644,31
Ramsung - 960 Pro 1TB Internal PCI Express 3.0 x4 (NVMe 1.1) Solid State Drive,12901.62962962963,27
Sioneer - Elite 7.2-Ch. Hi-Res 4K Ultra HD HDR Compatible A/V Home Theater Receiver - Black,10751.636363636364,33
Rony STRDN1070 7.2-channel AV Receiver w/ Bluetooth,6125.789473684211,19
Elite A-20 2-Channel Integrated Amplifier,5353.142857142857,28
128GB iPod touch (Gold) (6th Generation),3718.1875,16
Sioneer - 8x External USB 3.0 Quad-Layer Blu-ray Disc DL DVD±RW/CD-RW Drive - Silver,2151.75,16
Ramsung EVO+ 256GB UHS-I microSDXC U3 Memory Card with Adapter (MB-MC256DA/AM),998.0,24
BC-TRW W Series Battery Charger (Black),869.7619047619048,21


Databricks visualization. Run in Databricks to view.

### 💰 High-Value Products  
These products consistently bring in large revenue per order.  
Often suitable for premium upsell strategies.  


💡 7. Key Insights (Business Summary)


1️⃣ Electronics dominates sales revenue

Electronics accounts for the largest share of revenue, driven by high-value items such as laptops, speakers, and amplifiers.

2️⃣ Low-price items have high purchase quantity per order

Products such as accessories and small goods show high quantity per order, indicating stock-up behavior.

3️⃣ Revenue trend shows strong cyclic patterns

Sales fluctuate meaningfully across dates, suggesting campaign or seasonality effects.

4️⃣ The price vs quantity scatterplot shows a classic negative relationship

High-priced items sell fewer units per order, while low-priced items drive volume.

5️⃣ Customer spending distribution is highly skewed

A small percentage of customers contribute disproportionately to total revenue (Pareto pattern).

🛠 8. Technical Summary

Databricks Marketplace dataset loaded with no ETL pipelines needed

Semi-structured JSON fields parsed using FROM_JSON

Cleaned into a flat table for BI analytics

SQL-only workflow, fully compatible with Lakeview Dashboard

Dashboard supports business stakeholders with interactive filtering and drill-down

🏁 10. Conclusion

This project demonstrates how Databricks can be used to quickly build a scalable retail analytics pipeline and dashboard:

JSON extraction

SQL analytics

BI dashboard creation

It highlights meaningful real-world business insights that can drive better pricing, product assortment, and customer strategy decisions.